In [1]:
import pandas as pd

A = pd.read_csv("data/A.csv").values
import numpy as np

x = np.array(A[:, 0:3].astype(np.float32))
x = np.hstack([x, np.ones([len(x[:, 0]), 1])])

y = np.array(A[:, 3:5].astype(np.float32))
from homography import Transformer

t = Transformer()
t.fit_homography(x, y)
import pickle

with open('./transformer.pickle', 'wb') as f:
    pickle.dump(t, f)

In [2]:
import pandas as pd

A = pd.read_csv("data/units_6100_gt.csv")

In [3]:
camera = A.loc[A['name']=='Camera']
c_x, c_y = float(camera['pos.x']), float(camera['pos.y'])

In [4]:
A.drop(A[A['name']=='Camera'].index, inplace=True)

In [5]:
A['pos.x'] -= c_x
A['pos.y'] -= c_y

In [6]:
from homography import Transformer
import pickle

with open("./transformer.pickle", 'rb') as f:
    t = pickle.load(f)

uv = t.homography_transform(A[['pos.x', 'pos.y', 'pos.z']].values)

In [7]:
A['u'] = uv[:, 0]
A['v'] = uv[:, 1]

In [8]:
A['w'] = A['radius']*t.x_scale
A['h'] = A['radius']*t.y_scale

In [9]:
yolo_data = A[['unit_type', 'u', 'v', 'w', 'h']].values

In [11]:
import pybboxes as pbx
import cv2
import matplotlib.pyplot as plt

img = cv2.imread("data/tick_6100.png")

for box in yolo_data.tolist():
    b = pbx.YoloBoundingBox(box[1], box[2], box[3], box[4], (t.image_size, t.image_size))
    b = pbx.convert_bbox(b, to_type="voc")
    cv2.rectangle(img, (b[0], b[1]), (b[2], b[3]), (0, 0, 255), 1)


cv2.imwrite("tick_6100_boxes.png", img)

True